In [3]:
!pip freeze > aaa.txt

In [1]:
## !pip install -r requirements.txt

In [1]:
import pandas as pd
import numpy as np

import nltk
import spacy

#from tqdm.auto import tqdm 
#tqdm.pandas()

In [2]:
from spacy.language import Language
from spacy_langdetect import LanguageDetector
from spacytextblob.spacytextblob import SpacyTextBlob

@Language.factory("language_detector")
def create_language_detector(nlp, name):
    return LanguageDetector(language_detection_function=None)


In [4]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0.tar.gz
nlp = spacy.load("en_core_web_sm")

nlp.add_pipe('sentencizer')
nlp.add_pipe('language_detector')
nlp.add_pipe('spacytextblob')

     |████████████████████████████████| 13.9 MB 3.6 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for en-core-web-sm: filename=en_core_web_sm-3.2.0-py3-none-any.whl size=13900222 sha256=e847b0db78f220f3bb8c861b139dbc6949d58a6273534076836c02226359c1fd
  Stored in directory: /home/jovyan/.cache/pip/wheels/6b/f3/e3/63027c0c01b21850a0b2ca7708201d27b9bbfbbeeeaa0b9b92
Successfully built en-core-web-sm


In [5]:
df = pd.read_csv(r"https://www.ecb.europa.eu/press/key/shared/data/all_ECB_speeches.csv?848ea64ce6d77827b5e8e18790878b64",index_col = ["date"] , sep = "|",encoding='utf-8')

In [6]:
df = df.dropna()
df = df[df["contents"]!=" "]
df.sort_values("contents")

,speakers,title,subtitle,contents
date,,,,
2009-12-11,Gertrude Tumpel-Gugerell,"The Way Forward with Monetary, Fiscal and Mac...","Speech by Gertrude Tumpel-Gugerell, Member of ...",The Way Forward with Monetar...
2003-05-08,Eugenio Domingo Solans,Catalunya dins l'Europa moderna,"Eugenio Domingo Solans, Membre del Consell de ...",Catalunya dins l'Europa moderna Eugenio Do...
2017-12-06,Yves Mersch,Challenges for euro area monetary policy in e...,"Speech by Yves Mersch, Member of the Executive...",Challenges for euro area monetary policy in...
2019-10-17,Luis de Guindos,Speaking notes on climate-related risks,"Speaking notes by Luis de Guindos, Vice-Presid...","SPEAKING NOTES Washington, D.C., 17 October..."
2020-12-14,Fabio Panetta,A commitment to the recovery,"Speech by Fabio Panetta, Member of the Executi...",SPEECH A commitment to the recovery Spe...
...,...,...,...,...
2013-05-17,Yves Mersch,“Built to Last”: The New Euro Area Framework,"Speech by Yves Mersch, Member of the Executive...",“Built to Last”: The New Euro Area Framework...
2014-01-13,Yves Mersch,“Europe after the warm reboot”,"Speech by Yves Mersch, Member of the Executive...",“Europe after the warm reboot” Speech by ...
2015-11-16,Vítor Constâncio,“Monetary policy and the euro area problem”,"Speech by Vítor Constâncio, Vice-President of ...",“Monetary policy and the euro area problem” ...


In [ ]:
### /!\ Prend beaucoup de temps

df["nlp"] = df["contents"].apply(lambda x: nlp(x))

In [ ]:
df= df[df['nlp'].apply(lambda x: x._.language['language']=="en")]

In [ ]:
df["sentences"] = df["nlp"].apply(lambda y : [str(x) for x in list(y.sents)])

In [ ]:
# df = df.drop(columns=['contents', 'nlp'])

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [ ]:
def finbert_result(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    outputs = finbert(**inputs)[0]
    score = {0:0, 1:1, 2:-1}
    return np.vectorize(lambda x: score[x])(np.argmax(outputs.detach().numpy(), axis = 1)).mean()

In [ ]:
finbert_result(df["finbertscore"].iloc[0])

In [6]:
df

NameError: name 'df' is not defined

In [20]:
sentences = ["I represent the BCE", 
             "the sky is grey", 
             "the sky is blue", 
             "I am happy"]

inputs = tokenizer(sentences, return_tensors="pt", padding=True)
outputs = finbert(**inputs)[0]

labels = {0:'neutral', 1:'positive',2:'negative'}
for idx, sent in enumerate(sentences):
    print(sent, '----', outputs.detach().numpy()[idx])

I represent the BCE ---- [ 5.89971   -6.314463  -3.9419703]
the sky is grey ---- [ 6.2517743 -7.200073  -2.525556 ]
the sky is blue ---- [ 6.007808  -4.8922215 -3.1983948]
I am happy ---- [-3.57437    7.118817  -5.7525463]


In [23]:
score = {0:0, 1:1, 2:-1}
np.vectorize(lambda x: score[x])(np.argmax(outputs.detach().numpy(), axis = 1)).mean()

0.25

In [ ]:
[nlp(x)._.polarity for x in text]

In [21]:
outputs

tensor([[ 5.8997, -6.3145, -3.9420],
        [ 6.2518, -7.2001, -2.5256],
        [ 6.0078, -4.8922, -3.1984],
        [-3.5744,  7.1188, -5.7525]], grad_fn=<AddmmBackward0>)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(stop_words="english")
tfidf_vector = v.fit_transform(df["nlp"].apply(lambda x: x.text))


In [ ]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=df.index, columns=v.get_feature_names())
tfidf_df = tfidf_df.stack().reset_index()
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})
tfidf_df.sort_values(by=['date','tfidf'], ascending=[True,False]).groupby(['date']).head(10)


In [ ]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize("happiness")

In [ ]:
def clean_docs(texts, remove_stopwords=True, n_process = 4):
    nlp = spacy.load("en_core_web_sm")
    docs = nlp.pipe(texts, 
                    disable=['parser','lemmatizer' ,'ner','textcat'])
    stopwords = nlp.Defaults.stop_words

    docs_cleaned = []
    for doc in docs:
        tokens = [tok.text.lower().strip() for tok in doc if not tok.is_punct]
        if remove_stopwords:
            tokens = [ps.stem(tok) for tok in tokens if tok not in stopwords]
        doc_clean = ' '.join(tokens)
        docs_cleaned.append(doc_clean)
        
    return docs_cleaned

In [ ]:
aaa = df["contents"].sample(1)
cleaned = clean_docs(aaa, remove_stopwords=True)
cleaned

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
tfidf_vector = v.fit_transform(clean_docs(df["contents"]))
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=df.index, columns=v.get_feature_names())
tfidf_df = tfidf_df.stack().reset_index()
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})
tfidf_df.sort_values(by=['date','tfidf'], ascending=[True,False]).groupby(['date']).head(10)


In [ ]:
tfidf_vector = v.fit_transform(clean_docs(df["contents"]))

In [ ]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=df.index, columns=v.get_feature_names())
tfidf_df = tfidf_df.stack().reset_index()


In [ ]:
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})


In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
ps = PorterStemmer()
 
# choose some words to be stemmed
words = ["I", "am","happy","but","he","is","not"]
for word in words:
    print(ps.stem(lemmatizer.lemmatize(word)))

In [ ]:
tfidf_df = tfidf_df.sort_values(by=['date','tfidf'], ascending=[True,False]).groupby(['date']).head(10)
tfidf_df